In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Dell\\Desktop\\Chicken\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Dell\\Desktop\\Chicken\\Chicken-Disease-Classification'

In [36]:
from dataclasses import dataclass
from pathlib import Path

@dataclass (frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [37]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [38]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        config_filepath = Path.cwd() / config_filepath
        params_filepath = Path.cwd() / params_filepath
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [39]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import  logger
from cnnClassifier.utils.common import get_size
import shutil
import gzip

In [40]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config 


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download with following info : \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extraxt_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [43]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    # data_ingestion.extraxt_zip_file()
except Exception as e:
    raise e

[2023-07-15 12:01:44,141: INFO: common: yaml file : c:\Users\Dell\Desktop\Chicken\Chicken-Disease-Classification\config\config.yaml is successfully added:]
[2023-07-15 12:01:44,149: INFO: common: yaml file : c:\Users\Dell\Desktop\Chicken\Chicken-Disease-Classification\params.yaml is successfully added:]
['artifacts']
[2023-07-15 12:01:44,149: INFO: common: directory : artifacts is successfully created:]
['artifacts/data_ingestion']
[2023-07-15 12:01:44,149: INFO: common: directory : artifacts/data_ingestion is successfully created:]
[2023-07-15 12:01:44,157: INFO: 1420150900: file already exists of size: ~ 116 KB:]
